In [ ]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install jsonlines --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00


In [ ]:
from transformers import (
    BertTokenizer,
    BertModel,
    TFBertModel,
    BertConfig,
    AutoTokenizer,
    AutoModel,
)

In [ ]:
import pandas as pd
import numpy as np
import random
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = '/content/drive/MyDrive/w266_final_project/data/GoodNewsEveryone'

In [ ]:
DATA_PATH_EMOCAUSE = '/content/drive/MyDrive/w266_final_project/data/EmoCause'

In [ ]:
DATA_PATH_LOCAL = './data/GoodNewsEveryone'

In [ ]:
cause_model_name = 'mrm8488/spanbert-finetuned-squadv2'

spanbert_tokenizer = BertTokenizer.from_pretrained(cause_model_name)
spanbert_model = TFBertModel.from_pretrained(cause_model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
vocab_size = 28996

In [ ]:
bert_vocab = spanbert_tokenizer.convert_ids_to_tokens([i for i in range(vocab_size)])

In [ ]:
tokenized_vocab = spanbert_tokenizer(bert_vocab,
                                     max_length=3,
                                     truncation=True,
                                     padding='max_length',
                                     return_tensors='tf')

In [ ]:
token_embeds = spanbert_model(tokenized_vocab)[0][:,1,:]

In [ ]:
def find_synonyms_multi(input, topn = 1):
  tokenized_input = spanbert_tokenizer(input,
                                      max_length=3,
                                      truncation=True,
                                      padding='max_length',
                                      return_tensors='tf')

  input_vec = spanbert_model(tokenized_input)[0][:,1,:]

  input_dot_embeds = np.dot(input_vec, np.transpose(token_embeds))
  input_dot_input = np.repeat(np.expand_dims(np.sum(input_vec*input_vec, axis=1),
                                            axis=1),
                              vocab_size,
                              axis=1)
  embed_dot_embed = np.sum(token_embeds*token_embeds, axis=1)

  cosine_sims = input_dot_embeds / np.sqrt(input_dot_input * embed_dot_embed)

  topn_tokens = np.argsort(-cosine_sims, axis=1)[:,1:1+topn]

  return topn_tokens, np.apply_along_axis(spanbert_tokenizer.convert_ids_to_tokens, 1, topn_tokens)

In [ ]:
bert_syn_dict = {}
for i in range(14):
  syn_ids = find_synonyms_multi(bert_vocab[2000*i:2000*(i+1)])[0].squeeze()
  bert_syn_dict_batch = {2000*i + idx:e for (idx, e) in enumerate(syn_ids)}
  bert_syn_dict.update(bert_syn_dict_batch)

In [ ]:
syn_ids = find_synonyms_multi(bert_vocab[28000:28996])[0].squeeze()
bert_syn_dict_batch = {28000 + idx:e for (idx, e) in enumerate(syn_ids)}
bert_syn_dict.update(bert_syn_dict_batch)

#### Random Synonym Swap

In [ ]:
# Random token swap procedure:
# Grab a random row of the dataframe
# Take the tokens of the original situation
# Randomly select (without replacement) 1-5 words among the tokens
# Replace the tokens
# Add a new row to that dataframe

In [ ]:
gne_train_preprocessed = pd.read_csv(f'{DATA_PATH}/gne_train_preprocessed.csv')

In [ ]:
emocause_train_preprocessed = pd.read_csv(f'{DATA_PATH_EMOCAUSE}/emocause_train_preprocessed.csv')

In [ ]:
gne_train_preprocessed

,Unnamed: 0,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
0,3466,delta airlines refuses to boycott georgia over...,disgust,"[[""to boycott georgia over abortion laws""]]",to boycott georgia over abortion laws,4,11,2
1,1848,french special forces rescue hostages in a dea...,joy,"[[""rescue hostages in a deadly burkina faso ra...",rescue hostages in a deadly burkina faso raid,5,17,5
2,3195,"palestinian government sworn in, again, after ...",negative_surprise,"[[""sworn in, again, after botched oath""]]","sworn in, again, after botched oath",5,14,8
3,3683,cleric: muslims ‘have a duty’ to obtain nuclea...,negative_anticipation_including_pessimism,"[[""to obtain nuclear weapons""]]",to obtain nuclear weapons,14,17,7
4,194,usa today sports attacks on 'raging' trump app...,disgust,"[[""appear to be coordinated media campaign""]]",appear to be coordinated media campaign,12,17,2
...,...,...,...,...,...,...,...,...
3067,4777,the big drop in corporate tax rates and revenues,negative_surprise,"[[""tax rates and revenues""]]",tax rates and revenues,6,9,8
3068,2018,letters: maybe dan fagan should read hemingway...,positive_anticipation_including_optimism,"[[""maybe dan fagan should read hemingway and t...",maybe dan fagan should read hemingway and thoreau,3,16,9
3069,2557,"pope in morocco: walls, fear-mongering won't s...",fear,"[[""walls, fear-mongering won't stop migration""]]","walls, fear-mongering won't stop migration",7,17,3
3070,3161,house republicans ask democrats to help force ...,positive_surprise,"[[""help force vote on protecting lives of babi...",help force vote on protecting lives of babies ...,8,18,10


In [ ]:
emocause_train_preprocessed = emocause_train_preprocessed.rename(columns={'original_situation': 'headline', 'emotion': 'dominant_emotion'})
emocause_train_preprocessed

,Unnamed: 0,headline,dominant_emotion,cause_span,start_index,end_index,emotion_label
0,3651,one time i saw a giant cockroach under my desk!,__surprised__,giant cockroach,6,8,29
1,3375,i started a diet today and as soon as i got ho...,__guilty__,diet today and as soon as i got home i ate a b...,4,17,18
2,2609,i was late for a meeing.,__terrified__,late,3,3,30
3,308,i was buying some beer and smokes with a coupl...,__surprised__,full blown speakeasy,69,73,29
4,287,my husband was ordained to be a minister last ...,__proud__,husband was ordained to be a minister last yea...,2,27,26
...,...,...,...,...,...,...,...
3015,459,my wife and i had to move far away from home r...,__grateful__,cosign,25,26,17
3016,317,i had been working on my dance moves for the l...,__confident__,working on my dance,4,7,8
3017,1385,"when i went into labor with my youngest child,...",__terrified__,heartbeat dropped,22,23,30
3018,3026,last week there was a large spider on my wall.,__afraid__,large spider on my wall,6,10,0


In [ ]:
emocause_train_remapped = emocause_train_preprocessed

In [ ]:
emocause_train_remapped.dominant_emotion = emocause_train_remapped.dominant_emotion.map({'__angry__':	'anger',
                                                                                         '__jealous__':	'anger',
                                                                                         '__furious__':	'anger',
                                                                                         '__annoyed__':	'annoyance',
                                                                                         '__disgusted__':	'disgust',
                                                                                         '__afraid__':	'fear',
                                                                                         '__terrified__':	'fear',
                                                                                         '__guilty__':	'guilt',
                                                                                         '__joyful__':	'joy',
                                                                                         '__sentimental__':	'love_including_like',
                                                                                         '__content__':	'love_including_like',
                                                                                         '__nostalgic__':	'love_including_like',
                                                                                         '__caring__':	'love_including_like',
                                                                                         '__grateful__':	'love_including_like',
                                                                                         '__faithful__':	'love_including_like',
                                                                                         '__anxious__':	'negative_anticipation_including_pessimism',
                                                                                         '__apprehensive__':	'negative_anticipation_including_pessimism',
                                                                                         '__embarrassed__':	'negative_surprise',
                                                                                         '__disappointed__':	'negative_surprise',
                                                                                         '__devastated__':	'negative_surprise',
                                                                                         '__excited__':	'positive_anticipation_including_optimism',
                                                                                         '__prepared__':	'positive_anticipation_including_optimism',
                                                                                         '__anticipating__':	'positive_anticipation_including_optimism',
                                                                                         '__hopeful__':	'positive_anticipation_including_optimism',
                                                                                         '__surprised__':	'positive_surprise',
                                                                                         '__impressed__':	'positive_surprise',
                                                                                         '__proud__':	'pride',
                                                                                         '__confident__':	'pride',
                                                                                         '__sad__':	'sadness',
                                                                                         '__lonely__':	'sadness',
                                                                                         '__ashamed__':	'shame',
                                                                                         '__trusting__':	'trust'})

emocause_train_remapped.emotion_label = emocause_train_remapped.dominant_emotion.map({'anger':0,
                                                                                      'annoyance':1,
                                                                                      'disgust':2,
                                                                                      'fear':3,
                                                                                      'guilt':4,
                                                                                      'joy':5,
                                                                                      'love_including_like':6,
                                                                                      'negative_anticipation_including_pessimism':7,
                                                                                      'negative_surprise':8,
                                                                                      'positive_anticipation_including_optimism':9,
                                                                                      'positive_surprise':10,
                                                                                      'pride':11,
                                                                                      'sadness':12,
                                                                                      'shame':13,
                                                                                      'trust':14})
emocause_train_remapped

,Unnamed: 0,headline,dominant_emotion,cause_span,start_index,end_index,emotion_label
0,3651,one time i saw a giant cockroach under my desk!,positive_surprise,giant cockroach,6,8,10
1,3375,i started a diet today and as soon as i got ho...,guilt,diet today and as soon as i got home i ate a b...,4,17,4
2,2609,i was late for a meeing.,fear,late,3,3,3
3,308,i was buying some beer and smokes with a coupl...,positive_surprise,full blown speakeasy,69,73,10
4,287,my husband was ordained to be a minister last ...,pride,husband was ordained to be a minister last yea...,2,27,11
...,...,...,...,...,...,...,...
3015,459,my wife and i had to move far away from home r...,love_including_like,cosign,25,26,6
3016,317,i had been working on my dance moves for the l...,pride,working on my dance,4,7,11
3017,1385,"when i went into labor with my youngest child,...",fear,heartbeat dropped,22,23,3
3018,3026,last week there was a large spider on my wall.,fear,large spider on my wall,6,10,3


In [ ]:
append_emocause = True

if append_emocause:
  gne_train_preprocessed = pd.concat([gne_train_preprocessed, emocause_train_remapped])

In [ ]:
gne_train_preprocessed

,Unnamed: 0,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
0,3466,delta airlines refuses to boycott georgia over...,disgust,"[[""to boycott georgia over abortion laws""]]",to boycott georgia over abortion laws,4,11,2
1,1848,french special forces rescue hostages in a dea...,joy,"[[""rescue hostages in a deadly burkina faso ra...",rescue hostages in a deadly burkina faso raid,5,17,5
2,3195,"palestinian government sworn in, again, after ...",negative_surprise,"[[""sworn in, again, after botched oath""]]","sworn in, again, after botched oath",5,14,8
3,3683,cleric: muslims ‘have a duty’ to obtain nuclea...,negative_anticipation_including_pessimism,"[[""to obtain nuclear weapons""]]",to obtain nuclear weapons,14,17,7
4,194,usa today sports attacks on 'raging' trump app...,disgust,"[[""appear to be coordinated media campaign""]]",appear to be coordinated media campaign,12,17,2
...,...,...,...,...,...,...,...,...
3015,459,my wife and i had to move far away from home r...,love_including_like,NaN,cosign,25,26,6
3016,317,i had been working on my dance moves for the l...,pride,NaN,working on my dance,4,7,11
3017,1385,"when i went into labor with my youngest child,...",fear,NaN,heartbeat dropped,22,23,3
3018,3026,last week there was a large spider on my wall.,fear,NaN,large spider on my wall,6,10,3


In [ ]:
balance_class_samples = math.ceil(20000 / len(np.unique(gne_train_preprocessed.dominant_emotion))) - gne_train_preprocessed.dominant_emotion.value_counts()

In [ ]:
dict(balance_class_samples)

{'negative_surprise': 477,
 'positive_surprise': 711,
 'anger': 767,
 'positive_anticipation_including_optimism': 769,
 'love_including_like': 773,
 'fear': 881,
 'sadness': 894,
 'negative_anticipation_including_pessimism': 939,
 'disgust': 1002,
 'annoyance': 1021,
 'joy': 1077,
 'pride': 1101,
 'guilt': 1153,
 'shame': 1172,
 'trust': 1181}

In [ ]:
augment = pd.DataFrame()
for k in dict(balance_class_samples):
  augment = pd.concat([augment, gne_train_preprocessed[gne_train_preprocessed.dominant_emotion == k].sample(balance_class_samples[k], replace=True)])

In [ ]:
MAX_SEQUENCE_LENGTH = 124
augment_sit_encodings = spanbert_tokenizer(list(augment.headline),
                                           padding='max_length',
                                           max_length=MAX_SEQUENCE_LENGTH,
                                           return_tensors='tf')

In [ ]:
end_of_sit = [np.argmax(i == 0) - 1 for i in augment_sit_encodings.input_ids]

In [ ]:
for i in range(len(end_of_sit)):
  if end_of_sit[i] == -1:
    end_of_sit[i] = 123

In [ ]:
random_ind_select = []
for i in range(len(augment)):
  ind_list = random.sample(range(1, end_of_sit[i]), random.sample(range(1, min(5, end_of_sit[i])), 1)[0])
  random_ind_select.append(ind_list)

In [ ]:
full_tokens_replace = []

for i in range(len(list(augment_sit_encodings['input_ids']))):
    full_tokens_replace.append(list(np.take(augment_sit_encodings['input_ids'][i], random_ind_select[i])))

In [ ]:
# Replace the tokens
new_headlines = []

for i in range(len(random_ind_select)):
  a = augment_sit_encodings['input_ids'][i].numpy()
  np.put(a, random_ind_select[i], [bert_syn_dict[id] for id in full_tokens_replace[i]])

  new_headlines.append(spanbert_tokenizer.convert_tokens_to_string(spanbert_tokenizer.convert_ids_to_tokens(a[1:end_of_sit[i]])))

In [ ]:
augment['headline'] = new_headlines

In [ ]:
augment

,Unnamed: 0,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
2515,3734,i just guarded my pet dog . i do not know what...,negative_surprise,NaN,lost my pet dog,3,6,8
1005,4741,"boy , 14 , is airlifted to hospital after a tr...",negative_surprise,"[[""tree falls on him while he walks into schoo...",tree falls on him while he walks into school,13,21,8
2837,2503,i left the bathroom with a bit of poo in my pa...,negative_surprise,NaN,poo in my pants once. i was in 3rd grade and s...,9,26,8
992,426,he was my birthday today ! i came home from wo...,negative_surprise,NaN,birthday today ! i came home from work to find...,4,19,8
1462,3870,real sanctuary demands more naive lip service ...,negative_surprise,"[[""more than lip service. cities must step up.""]]",more than lip service. cities must step up.,4,13,8
...,...,...,...,...,...,...,...,...
729,88,"whenever we ' ve moved abroad , i ' ve learned...",trust,NaN,instinct and sense of adventure,15,19,14
603,1413,federal judge upholds rerump 278 - term health...,trust,"[[""short-term health plan expansion""]]",short-term health plan expansion,8,13,14
1986,2792,m didnfred feel bucket going there,trust,NaN,feel comfortable going,4,6,14
431,3396,that the arcade,trust,NaN,universe,3,3,14


In [ ]:
augment_2_gne_r_balanced = pd.concat([gne_train_preprocessed, augment])
augment_2_gne_r_balanced = augment_2_gne_r_balanced.sample(frac=1)
augment_2_gne_r_balanced

,Unnamed: 0,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
513,270,doctors use medical records in track down unva...,trust,"[[""track down unvaccinated""]]",track down unvaccinated,6,11,14
322,3894,war they mran would be even gotten disastrous ...,negative_anticipation_including_pessimism,"[[""would be even more disastrous than iraq""]]",would be even more disastrous than iraq,5,13,7
2425,4424,poll : commerce ‘ most important Problem ’ fac...,fear,"[[""immigration ‘most important problem’ facing...",immigration ‘most important problem’ facing us,3,10,3
30,3253,us border procedure is the most brutal militia...,fear,"[[""us border patrol is the most brutal militia...",us border patrol is the most brutal militia of...,1,10,3
974,1996,pricey pension rescue grouped to house Floor N...,positive_surprise,"[[""house floor next week""]]",house floor next week,7,10,10
...,...,...,...,...,...,...,...,...
2386,4166,armed militia detained hundreds for migrants i...,shame,"[[""detained hundreds of migrants in new mexico""]]",detained hundreds of migrants in new mexico,3,11,13
2387,2639,latest fundraising numbers from beto o’rourke ...,disgust,"[[""beto o’rourke""]]",beto o’rourke,5,11,2
2077,2994,inmates rescue cottage partially outside car,pride,"[[""rescue baby locked inside car""]]",rescue baby locked inside car,2,6,11
511,2361,trump’s racist comments have spawned violent t...,anger,"[[""racist comments have spawned""]]",racist comments have spawned,5,8,0


In [ ]:
from google.colab import files
augment_2_gne_r_balanced.to_csv('augment_2_gne_r_balanced.csv')
files.download('augment_2_gne_r_balanced.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Random Token Insertion

In [ ]:
def insert_random_tokens(text, start_ind, end_ind):

  # Tokenize input text and grab end index.
  text_encoding = spanbert_tokenizer(text,
                                     padding=True,
                                     return_tensors='tf')

  end_of_text = text_encoding.input_ids[0].shape[0]

  # We randomly choose between 1 and num_tokens / 8 spots to insert tokens in.
  num_insertions = random.randint(1, max(1, math.ceil(16 / 8)))

  text_ids = text_encoding.input_ids[0]
  new_start_ind = start_ind
  new_end_ind = end_ind

  for i in range(num_insertions):
    # Choose a random position to insert token in.
    insertion_index = random.randint(1, end_of_text - 2)
    syn_sel_index = random.randint(1, end_of_text - 2)

    # If the token is on or before start index, add 1 to both start and end indices.
    if insertion_index <= start_ind:
      new_start_ind += 1
      new_end_ind += 1
    # If it is after start index and on/before end index, add 1 to end index only.
    elif insertion_index > start_ind and insertion_index <= end_ind:
      new_end_ind += 1
    else:
      pass

    id_ss_ind = text_encoding.input_ids.numpy()[0][syn_sel_index]

    # New token takes the place of position and moves everything to the right.
    # New token will be a synonym of random token in sentence.
    text_ids = np.insert(text_ids, insertion_index, bert_syn_dict[id_ss_ind])
    end_of_text += 1

    # Note that in this method we could add synonyms of augmented synonyms.

  new_text = spanbert_tokenizer.convert_tokens_to_string(spanbert_tokenizer.convert_ids_to_tokens(text_ids[1:end_of_text - 1]))
  return new_text, new_start_ind, new_end_ind

In [ ]:
augment_next = augment

In [ ]:
augment_next[['headline', 'start_index', 'end_index']] = pd.DataFrame(augment_next.apply(lambda x: insert_random_tokens(x['headline'], int(x['start_index']), int(x['end_index'])), axis=1, result_type='expand'))

In [ ]:
augment_next

,Unnamed: 0,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
2515,3734,i just guarded my pet dog . i do m not know wh...,negative_surprise,NaN,lost my pet dog,3,6,8
1005,4741,"boy , 14 , is airlifted to hospital after a tr...",negative_surprise,"[[""tree falls on him while he walks into schoo...",tree falls on him while he walks into school,13,23,8
2837,2503,i left the bathroom withfred a bit of poo an i...,negative_surprise,NaN,poo in my pants once. i was in 3rd grade and s...,10,28,8
992,426,he was my birthday today ! i came home from wo...,negative_surprise,NaN,birthday today ! i came home from work to find...,4,19,8
1462,3870,real sanctuary demands more naive lip service ...,negative_surprise,"[[""more than lip service. cities must step up.""]]",more than lip service. cities must step up.,4,14,8
...,...,...,...,...,...,...,...,...
729,88,"whenever we ' ve ... moved abroad , i ' ve lea...",trust,NaN,instinct and sense of adventure,17,21,14
603,1413,federal judge fled upholds rerumpfred 278 - te...,trust,"[[""short-term health plan expansion""]]",short-term health plan expansion,10,15,14
1986,2792,m didnfred feel bucket feeling going there,trust,NaN,feel comfortable going,4,7,14
431,3396,that become the arcade,trust,NaN,universe,4,4,14


### Random Token Deletion

In [ ]:
first_ex = gne_train_preprocessed.iloc[0]
first_ex

Unnamed: 0                                                       3466
headline            delta airlines refuses to boycott georgia over...
dominant_emotion                                              disgust
cause                     [["to boycott georgia over abortion laws"]]
cause_span                      to boycott georgia over abortion laws
start_index                                                         4
end_index                                                          11
emotion_label                                                       2
Name: 0, dtype: object

In [ ]:
first_ex['headline']

'delta airlines refuses to boycott georgia over abortion laws'

In [ ]:
'''
text: The raw string to process

start_ind: The start index of the target span (indexing starts from 1 not 0)

end_ind: The end index of the target span (indexing starts from 1 not 0)

tokenizer: The tokenizer object to use to tokenize the input

p_delete: Probability that we delete a token

inside_target_range: If set, will allow for random deletion of tokens inside the target span (otherwise avoids them)
'''

def delete_random_tokens(text, start_ind,
                         end_ind, tokenizer=None,
                         p_delete=0.2, inside_target_range=False):

    if tokenizer == None:
        tokenizer = spanbert_tokenizer(text,
                                       padding=True,
                                       return_tensors='tf')

    # Convert index to start at 0 (instead of 1)
    start_ind -= 1
    end_ind -= 1

    # Determine number of tokens to delete
    tokens = tokenizer.tokenize(text)
    num_tokens = len(tokens)

    # Compute the range of indices to allow deletion
    allowed_indices = set(range(num_tokens))
    if inside_target_range:
        allowed_indices -= set(range(start_ind, end_ind + 1))

    # Ensure we delete at least one token
    num_tokens_to_delete = max(1, int(len(allowed_indices) * p_delete))

    # Randomly determine tokens for deletion
    deleted_indices = set(random.sample(allowed_indices, num_tokens_to_delete))

    # Process deleted indices to adjust start & end appropriately
    for i in sorted(deleted_indices, reverse=True):
        if i < start_ind:
            start_ind -= 1
            end_ind -= 1
        elif i == start_ind:
            start_ind += 1
        elif i == end_ind:
            end_ind -= 1

    # Convert index back to starting at 1 (instead of 0)
    start_ind += 1
    end_ind += 1

    # Copy over only tokens that were not "deleted"
    new_tokens = [token for i, token in enumerate(tokens) if i not in deleted_indices]
    new_text = tokenizer.convert_tokens_to_string(new_tokens)

    return new_text, start_ind, end_ind


#### Example: Run the random token deletion util on the same span 3 times

In [ ]:
text_with_deletion = []
new_indices = []
for _ in range(3):
    new_text, new_start, new_end = delete_random_tokens(first_ex['headline'],
                                                        first_ex['start_index'],
                                                        first_ex['end_index'],
                                                        spanbert_tokenizer)
    text_with_deletion.append(new_text)
    new_indices.append((new_start, new_end))

print(text_with_deletion)
print(new_indices)

['delta airlines to boycotteorgia over abortion laws', 'delta to boycott georgia over abortion laws', 'delta refuses to boycott georgia abortion laws']
[(3, 10), (2, 9), (3, 10)]


<ipython-input-74-1065fe1d9c4f>:41: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  deleted_indices = set(random.sample(allowed_indices, num_tokens_to_delete))


#### Example: Now allow target span to be included in deletion (we might not want to allow this for context)

In [ ]:
text_with_deletion = []
new_indices = []
for _ in range(3):
    new_text, new_start, new_end = delete_random_tokens(first_ex['headline'],
                                                        first_ex['start_index'],
                                                        first_ex['end_index'],
                                                        spanbert_tokenizer,
                                                        inside_target_range=True)
    text_with_deletion.append(new_text)
    new_indices.append((new_start, new_end))

print(text_with_deletion)
print(new_indices)

['airlines refuses to boycott georgia over abortion laws', 'delta airlines to boycott georgia over abortion laws', 'airlines refuses to boycott georgia over abortion laws']
[(3, 10), (3, 10), (3, 10)]


<ipython-input-74-1065fe1d9c4f>:41: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  deleted_indices = set(random.sample(allowed_indices, num_tokens_to_delete))


#### Apply to dataset

In [ ]:
augment_deletion = augment_next.apply(
    lambda row: delete_random_tokens(
        row['headline'],
        row['start_index'],
        row['end_index'],
        spanbert_tokenizer
    ),
    axis=1
)

<ipython-input-74-1065fe1d9c4f>:41: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  deleted_indices = set(random.sample(allowed_indices, num_tokens_to_delete))


In [ ]:
augment_deletion

2664    (psi their , aoc clash over reach of ‘s ’, 9, 10)
1974    (##ga - — opted protest told campus police she...
748     (##ibya may shut migrant detention net centres...
1294     (the trou regarded of martin luther king, 2, 14)
1130    (knew : public will ' on ’ congress impfred tr...
                              ...                        
2497    (nyt essay on a man ’ pleasant out journey on,...
2497      (On nyt essay on lawyer ‘ s car on guns, 4, 12)
2253    (man 800 stars couple 400 miles to his visit t...
452     (check sfredwa ' s classa all - state boys and...
1981    (in cofredfredstown hallfredy is remembered hi...
Length: 16938, dtype: object

#### Parse back into dataframe

In [ ]:
df_a_deletion = augment_next.copy()
df_a_deletion = df_a_deletion.reset_index(drop=True)

In [ ]:
# Populate the mutated columns
mutated_tuples = augment_deletion.values.tolist()

for i, row in df_a_deletion.iterrows():
    curr_tuple = mutated_tuples[i]
    text, start, end = curr_tuple

    df_a_deletion.at[i, 'headline'] = text
    df_a_deletion.at[i, 'start_index'] = str(start)
    df_a_deletion.at[i, 'end_index'] = str(end)

In [ ]:
df_augment_deletion = df_a_deletion

#### Compare the deleted strings and original strings

In [ ]:
rand_indices = random.sample([i for i in range(len(augment_next))], 5)
for i in rand_indices:
    before_str = augment_next['headline'].iloc[i]
    after_str = df_augment_deletion['headline'].iloc[i]
    print(f'BEFORE:\n\t{before_str}')
    print(f'AFTER:\n\t{after_str}')
    print('==========')

BEFORE:
	my girlfriend and i have birthdays 8 via days alongside . we exchanged gifts somewhere in the between and it made me so happy , it kind of felt like christmas in the summer !
AFTER:
	my and i have birthdays 8 via days . we exchanged gifts the between it made me happy , it kind of felt like christmas in summer !
BEFORE:
	utah man pleads not guilty in slayings of deputy at for jack in the box and russian snowboard champ that l . a .
AFTER:
	u man plea not guilty in slayings of for jack in the box and russian snowboard ch that l a .
BEFORE:
	i think i might loose your job over something that wasn ' t your m fault .
AFTER:
	i think i loose your job something that ' t your m fault .
BEFORE:
	sri startingnka blamefred westfreder bombings On local jihadis
AFTER:
	sri startingnka blamefred wester bombings localhadis
BEFORE:
	forthcoming had a to train a new customer on how in use our system [CLS] . i was readu and had an answer for them everything they threw at me .
AFTER:
	forthcomin

In [ ]:
augment_deletion = pd.concat([gne_train_preprocessed, df_augment_deletion])
augment_deletion = augment_deletion.sample(frac=1)
augment_deletion

,Unnamed: 0,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
2061,3281,"all 21 marvel superhero films, ranked",positive_surprise,"[[""marvel superhero""]]",marvel superhero,3,6,10
11879,3441,i bought something from the the lady gave me w...,shame,NaN,change,11,11,13
2984,2169,i left my room at night and a bat flew at me.,fear,NaN,bat flew,9,10,3
9114,2864,the weaponry alan club involves skiing On wate...,joy,"[[""competition involves skiing on water and sn...",competition involves skiing on water and snow,5,12,5
464,3763,m ' m really daughter in my ' extremely s acti...,negative_surprise,NaN,cheating,15,15,8
...,...,...,...,...,...,...,...,...
11011,2068,i should be health m and playing out instead i...,guilt,NaN,opposite,16,16,4
898,1203,trump was winner coming out of mueller hearing...,positive_surprise,"[[""coming out of mueller""],[""mueller hearings,...",coming out of mueller,5,10,10
12313,3441,i bought m something the and the lady gave me ...,shame,NaN,change,12,12,13
905,2112,i bought a used a few years and documented fou...,positive_surprise,NaN,found 500 dollars,10,12,10


In [ ]:
from google.colab import files
if not append_emocause:
  augment_deletion.to_csv('augment_3_gne_r_bal_delins.csv')
  files.download('augment_3_gne_r_bal_delins.csv')
else:
  augment_deletion.to_csv('augment_4_gne_w_emocause.csv')
  files.download('augment_4_gne_w_emocause.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>